# Moderate Image Using Azure Content Safety API

Start by installing the packages that will be required

In [ ]:
# install the packages
%pip install requests
%pip install azure-ai-contentsafety
%pip install python-dotenv

# Import Packages

Next, you'll import all the required packages

In [2]:
import os
from dotenv import load_dotenv
from azure.ai.contentsafety import ContentSafetyClient
from azure.ai.contentsafety.models import ImageCategory
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.ai.contentsafety.models import AnalyzeImageOptions, ImageData, AnalyzeImageOutputType

# Create Content Safety Client

You will create content safety `client` in this step which will be used to pass API requests to Azure servers for moderations 

In [3]:
# Load your Azure Safety API key and endpoint
load_dotenv()

key = os.environ["CONTENT_SAFETY_KEY"]
endpoint = os.environ["CONTENT_SAFETY_ENDPOINT"]

# Create a Content Safety client
client = ContentSafetyClient(endpoint, AzureKeyCredential(key))

# Moderate Image

Implement the `moderate_image` function

In [7]:
def moderate_image(image_data):
    # Construct a request
    request = AnalyzeImageOptions(image=ImageData(content=image_data))

    # Analyze image
    try:
        response = client.analyze_image(request)
    except HttpResponseError as e:
        print("Analyze image failed.")
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
            raise
        print(e)
        raise

    # Extract results
    categories = {
        ImageCategory.HATE: None,
        ImageCategory.SELF_HARM: None,
        ImageCategory.SEXUAL: None,
        ImageCategory.VIOLENCE: None
    }

    for item in response.categories_analysis:
        if item.category in categories:
            categories[item.category] = item

    hate_result = categories[ImageCategory.HATE]
    self_harm_result = categories[ImageCategory.SELF_HARM]
    sexual_result = categories[ImageCategory.SEXUAL]
    violence_result = categories[ImageCategory.VIOLENCE]

    # Check for inappropriate content
    violations = []
    if hate_result and hate_result.severity > 2:
        violations.append("hate speech")
    if self_harm_result and self_harm_result.severity > 3:
        violations.append("self-harm references")
    if sexual_result and sexual_result.severity > 0:
        violations.append("sexual references")
    if violence_result and violence_result.severity > 2:
        violations.append("violent references")

    if violations:
        return f"Your shared image contains {', '.join(violations)} that violate our community guidelines. Please modify your image to adhere to community guidelines."

    # If content is appropriate
    return "Post successful"

In [ ]:
with open('../sample-data/test-images/pexels-ash-craig-122861-376464.jpg', 'rb') as file:
    image_data = file.read()

moderation_response = moderate_image(image_data)
print(moderation_response)